In [20]:
import pandas as pd
import re

In [2]:
sentiments = pd.read_csv('./data/judge-1377884607_tweet_product_company.csv',encoding='cp850')

In [10]:
emotion  = 'is_there_an_emotion_directed_at_a_brand_or_product'
directed = 'emotion_in_tweet_is_directed_at'

In [19]:
sentiments.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,NaN,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion


In [22]:
sentiments.sample(5)['tweet_text']

3256    @mention gets a tattoo for a free ipad 2 {link...
7965    @mention says she feels sexier when she uses a...
5014    ë¸╝ Happy Woman's Day! Make love, not fuss! ë¸...
8249    Ad jg buat iphone RT @mention New #UberSocial ...
1510    Flipboard's next platform will be the iPhone, ...
Name: tweet_text, dtype: object

In [25]:
sentiments.iloc[5014]['tweet_text']

"ë¸╝ Happy Woman's Day! Make love, not fuss! ë¸_ {link} ëÒ_ #edchat #musedchat #sxsw #sxswi #classical #newTwitter"

In [5]:
for col in ['emotion_in_tweet_is_directed_at','is_there_an_emotion_directed_at_a_brand_or_product']:
    display(sentiments[col].value_counts())
sentiments.info()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [14]:
# sentiments.isna()
vals_ = sentiments[sentiments['emotion_in_tweet_is_directed_at'].isna()].value_counts('is_there_an_emotion_directed_at_a_brand_or_product')
vals_

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5298
Positive emotion                       306
I can't tell                           147
Negative emotion                        51
dtype: int64

In [18]:
sentiments[sentiments['tweet_text'].isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
6,NaN,NaN,No emotion toward brand or product


In [17]:
vals_.values.sum()

5802